In [ ]:
pip install bs4 selenium

In [ ]:
pip install webdriver-manager

In [ ]:
pip install --upgrade requests

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
# from lxml import etree
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [ ]:
driver.get("https://www.99.co/singapore/sale/hdb")
links=[]
count = 0
webcount = 0
first = True
while True:
  try:
    soup = BeautifulSoup(driver.page_source, 'lxml')
    #time.sleep(5)
    linkclass = soup.find_all('a', class_='_3Ajbv _30I97 _1vzK2')
    for l in linkclass:
      links.append(l['href'])
    nextbutton = "https://www.99.co" + soup.find('li', class_='next').find('a')['href']
  except:
    driver.refresh()
  else:
    while True:
      try:
        if count == 20:
            driver = webdriver.Chrome('chromedriver',options=chrome_options)
            count = 0
        driver.get(nextbutton)
        time.sleep(5)
      except:
         driver = webdriver.Chrome('chromedriver',options=chrome_options)
      else:
        count+=1
        webcount+=1
        break

  if webcount==228:
    break

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
t_page_source=[]
for link in links:
  if(count == 20):
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    count = 0
  else:
    pass
  driver.get("https://www.99.co" + link)
  while True:
  # time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "lxml")
    trans = soup.find(id="transactions")
    t_page_source.append(trans) 
    try:
      # nextbutton = soup.find('li', class_='RIVsn')
      t_nextbutton = driver.find_element(By.CLASS_NAME,"RIVsn")
      t_nextbutton.click()
      time.sleep(5)
      count+=1
    except:
      break
  time.sleep(10)

In [ ]:
for t in t_page_source:
  if t!= None:
    row1 = t.find_all('tr',class_="_3zkEn _1kKgW")
    row2 = t.find_all('tr',class_="_1kKgW")
    t_data = []
    for r in row1:
      t_data.append( r.find_all('td',class_="gcin4 _2kf8F"))
    for r in row2:
      t_data.append( r.find_all('td',class_="gcin4 _2kf8F"))
    for a in t_data:
      for r in range(len(a)):
        if(r==0):
          d = int(a[r].get_text()[3:])
        elif(r==1):
          b = a[r].get_text()
        elif(r==3):
          x = a[r].get_text().split(" ")
          af = int(x[0].replace(',', ''))
          am = float(x[2].replace(',', ''))
        elif(r==4):
          x = a[r].get_text().split(" ")
          if "M" in x[0][2:]:
            p = float(x[0][2:-1].replace(',', '')) * 10**6
          else:
            p = int(x[0][2:].replace(',', ''))
          pf = float(x[1][3:].replace(',', ''))
      df2=df2.concat({'Date' : d,'Block' : b,'Areasqft' : af, 'Areasqm' : am, 'Price' : p, 'Pricesf' : pf}, ignore_index=True)


In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
encoder = OrdinalEncoder()
df2['Block'] = encoder.fit_transform(df2[['Block']])
df2.info()

In [ ]:
df2.to_csv('residential.csv', index = False)